# Install dependencies


In [1]:
!pip install -qU torch==2.4.0 torchvision==0.19.0 "ray[client,train]"==2.34.0

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor
from tqdm import tqdm

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        for X, y in tqdm(train_dataloader, desc=f"Train Epoch {epoch}"):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader, desc=f"Test Epoch {epoch}"):
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [4]:
ray.init(
    address="ray://raycluster-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.4.0", "torchvision==0.19.0"]},
)

2024-08-20 15:04:06,335	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.34.0
    Python: 3.9.19
This process on Ray Client was started with:
    Ray: 2.34.0
    Python: 3.9.13



Python version:,3.9.19
Ray version:,2.34.0
Dashboard:,http://10.0.2.192:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote(num_workers=6, use_gpu=False))

(autoscaler +8s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +8s) Adding 1 node(s) of type small-group.
(autoscaler +8s) Resized to 4 CPUs.
(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) View detailed results here: /home/ray/ray_results/TorchTrainer_2024-08-20_08-07-16
(train_fashion_mnist pid=441, ip=10.0.2.84) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-08-20_07-55-50_885073_1/artifacts/2024-08-20_08-07-16/TorchTrainer_2024-08-20_08-07-16/driver_artifacts`
(autoscaler +3m16s) Adding 1 node(s) of type small-group.
(autoscaler +3m16s) Resized to 8 CPUs.
(autoscaler +3m16s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m21s) No available node types can fulfill resource reque

(train_fashion_mnist pid=441, ip=10.0.2.84) Ignore this message if the cluster is autoscaling. Training has not started in the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 7.0 CPUs and 0 GPUs, but the cluster only has 4.0 CPUs and 0 GPUs available. Stop the training and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


(autoscaler +4m12s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m17s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m22s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m27s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m32s) No available node types can fulfill resource requests {'bundle_group_66e7422e3d142a6d037037c6e2a102000000': 0.001}*1. Add suitable node types to this cluster to resolve this is

(RayTrainWorker pid=876, ip=10.0.2.84) Setting up process group for: env:// [rank=0, world_size=6]
(TorchTrainer pid=829, ip=10.0.2.84) Started distributed worker processes: 
(TorchTrainer pid=829, ip=10.0.2.84) - (node_id=5ae2bf3e104d858e8a50bde2092e640995c80f2dbaf1626944bad9dd, ip=10.0.2.84, pid=876) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=829, ip=10.0.2.84) - (node_id=5ae2bf3e104d858e8a50bde2092e640995c80f2dbaf1626944bad9dd, ip=10.0.2.84, pid=877) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=829, ip=10.0.2.84) - (node_id=5ae2bf3e104d858e8a50bde2092e640995c80f2dbaf1626944bad9dd, ip=10.0.2.84, pid=878) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=829, ip=10.0.2.84) - (node_id=0d576c11aafb62c15a9de755c70bb52bd0500d299da122e9022fef66, ip=10.0.2.66, pid=457) world_rank=3, local_rank=0, node_rank=1
(TorchTrainer pid=829, ip=10.0.2.84) - (node_id=0d576c11aafb62c15a9de755c70bb52bd0500d299da122e9022fef66, ip=10.0.2.66, pid=458) world_rank=4, local_

(RayTrainWorker pid=457, ip=10.0.2.66) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=877, ip=10.0.2.84) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=877, ip=10.0.2.84) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz
(RayTrainWorker pid=457, ip=10.0.2.66) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


...

(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) Training finished iteration 8 at 2024-08-20 08:14:27. Total running time: 7min 10s
(train_fashion_mnist pid=441, ip=10.0.2.84) ╭───────────────────────────────╮
(train_fashion_mnist pid=441, ip=10.0.2.84) │ Training result               │
(train_fashion_mnist pid=441, ip=10.0.2.84) ├───────────────────────────────┤
(train_fashion_mnist pid=441, ip=10.0.2.84) │ checkpoint_dir_name           │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_this_iter_s      28.6214 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_total_s          343.853 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ training_iteration          8 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ accuracy              0.86923 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ loss                  0.36118 │
(train_fashion_mnist pid=441, ip=10.0.2.84) ╰───────────────────────────────╯


Train Epoch 9:   0%|          | 0/2000 [00:00<?, ?it/s]


(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) Training finished iteration 9 at 2024-08-20 08:14:56. Total running time: 7min 39s
(train_fashion_mnist pid=441, ip=10.0.2.84) ╭───────────────────────────────╮
(train_fashion_mnist pid=441, ip=10.0.2.84) │ Training result               │
(train_fashion_mnist pid=441, ip=10.0.2.84) ├───────────────────────────────┤
(train_fashion_mnist pid=441, ip=10.0.2.84) │ checkpoint_dir_name           │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_this_iter_s      28.7684 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_total_s          372.621 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ training_iteration          9 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ accuracy              0.87762 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ loss                  0.35107 │
(train_fashion_mnist pid=441, ip=10.0.2.84) ╰───────────────────────────────╯


Test Epoch 9: 100%|██████████| 334/334 [00:00<00:00, 626.37it/s]


(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) Training finished iteration 10 at 2024-08-20 08:15:25. Total running time: 8min 8s
(train_fashion_mnist pid=441, ip=10.0.2.84) ╭───────────────────────────────╮
(train_fashion_mnist pid=441, ip=10.0.2.84) │ Training result               │
(train_fashion_mnist pid=441, ip=10.0.2.84) ├───────────────────────────────┤
(train_fashion_mnist pid=441, ip=10.0.2.84) │ checkpoint_dir_name           │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_this_iter_s      28.6079 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ time_total_s          401.229 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ training_iteration         10 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ accuracy              0.86743 │
(train_fashion_mnist pid=441, ip=10.0.2.84) │ loss                  0.35137 │
(train_fashion_mnist pid=441, ip=10.0.2.84) ╰───────────────────────────────╯


# Shut down Ray workers


In [6]:
ray.shutdown()

(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) Training completed after 10 iterations at 2024-08-20 08:15:26. Total running time: 8min 9s
(train_fashion_mnist pid=441, ip=10.0.2.84) 
(train_fashion_mnist pid=441, ip=10.0.2.84) Training result: Result(
(train_fashion_mnist pid=441, ip=10.0.2.84)   metrics={'loss': 0.35136826510797675, 'accuracy': 0.8674265146970606},
(train_fashion_mnist pid=441, ip=10.0.2.84)   path='/home/ray/ray_results/TorchTrainer_2024-08-20_08-07-16/TorchTrainer_e36e0_00000_0_2024-08-20_08-07-16',
(train_fashion_mnist pid=441, ip=10.0.2.84)   filesystem='local',
(train_fashion_mnist pid=441, ip=10.0.2.84)   checkpoint=None
(train_fashion_mnist pid=441, ip=10.0.2.84) )


(train_fashion_mnist pid=441, ip=10.0.2.84) Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/TorchTrainer_2024-08-20_08-07-16' in 0.0028s.
